In [56]:
# Model: https://huggingface.co/ukr-models/uk-ner

In [38]:
import random
import string
import torch
import transformers
import tokenize_uk

In [53]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

In [40]:
# source:
# https://blog.ehri-project.eu/uk/2023/05/05/the-holocaust-in-volhynia-uk/
# some phrases were changed to test edge cases
text = [
    "«У селі жили і поляки, і українці. Мамина рідна сестра вийшла заміж за поляка. Там вони живуть досі ті мої двоюрідні сестри. Батькові сестри також повиходили заміж за поляків. Поки війни не було, то [міжетнічної] різниці не було» – зазначає Ганна Кухарчук із с. Княгинінок.",
    "Про свій досвід взаємодії з євреями Антоніна Гук із с. Зелений Дуб пригадує наступне: «Жиди приходили до нас додому. Мама хотіли їм квасолі дати, а він каже «ми квасолі не їмо». Це так той єврей сказав. А чому він приходив, то я не знаю, мабуть через якусь потребу. Такі ж люди були, як і ми…»",
    "Потім він по-польськи сказав “Nemá nič”» – розповідає Галина Береза з с. Коршів Волинської області", # non-ukrainian phrases
    "Інший календар, яким часто послуговуються оповідачі – релігійний. Оскільки більшість етнічних українців Волині належать до православної конфесії, найважливішими святами для них є Різдво (7 січня), Водохреща (19 січня), Стрітення (15 лютого), Благовіщеня (7 квітня), Великдень (дата щороку змінюється), Трійця (через 50 днів після Великодня), Святих Петра і Павла (12 липня), Спаса (19 серпня) тощо. Всі ці свята відзначаються згідно юліанського (а не григоріанського) календаря.", # not a real entities
    "Людей не понищили, отак прийшли й зайняли територію. Батькам не було страшно, наш тато ні до кого не ліз, дбали тільки про свою сім’ю й ніхто не зачіпав. У 1939 р. було створено десь 10 чи 11 колгоспів. Бідні люди самі пішли в колгосп. Володимир теж пішов у колгосп. Йому платили по 9 кг. зерна за трудодень. Заробив він десь 40 метрів зерна (приблизно 4 тони)", # just first name
    "Тоді саме були жнива й Галина їх переховувала в копнах. Вона дуже багато євреїв переховувала…", # just first name
    "Ядвіга хотіли їм квасолі дати, а він каже «ми квасолі не їмо». Це так той єврей сказав.", # just first name
    "Цю думку ізраїльського історика цитує Джон-Пол Химка.", # non-ukrainian name
]    

In [42]:
tokenizer = AutoTokenizer.from_pretrained('ukr-models/uk-ner')
model = AutoModelForTokenClassification.from_pretrained('ukr-models/uk-ner')

/home/ubuntu/Projects/ua-testimonies/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [47]:
def get_word_predictions(model, tokenizer, texts, is_split_to_words=False, device='cpu'):
    words_res = []
    y_res = []

    if not is_split_to_words:
        texts = [tokenize_uk.tokenize_words(text) for text in texts]

    for text in texts:
        size = len(text)
        idx_list = [idx + 1 for idx, val in enumerate(text) if val in ['.', '?', '!']]
        if len(idx_list):
            sents = [text[i: j] for i, j in zip([0] + idx_list, idx_list + ([size] if idx_list[-1] != size else []))]
        else:
            sents = [text]

        y_res_x = []
        words_res_x = []
        for sent_tokens in sents:
            tokenized_inputs = [101]
            word_ids = [None]
            for word_id, word in enumerate(sent_tokens):
                word_tokens = tokenizer.encode(word)[1:-1]
                tokenized_inputs += word_tokens
                word_ids += [word_id]*len(word_tokens)
            tokenized_inputs = tokenized_inputs[:(tokenizer.model_max_length-1)]
            word_ids = word_ids[:(tokenizer.model_max_length-1)]
            tokenized_inputs += [102]
            word_ids += [None]

            torch_tokenized_inputs = torch.tensor(tokenized_inputs).unsqueeze(0)
            torch_attention_mask = torch.ones(torch_tokenized_inputs.shape)
            predictions = model.forward(input_ids=torch_tokenized_inputs.to(device), attention_mask=torch_attention_mask.to(device))
            predictions = torch.argmax(predictions.logits.squeeze(), axis=1).numpy()
            predictions = [model.config.id2label[i] for i in predictions]

            previous_word_idx = None
            sent_words = []
            predictions_words = []
            word_tokens = []
            first_pred = None
            for i, word_idx in enumerate(word_ids):
                if word_idx != previous_word_idx:
                    sent_words.append(tokenizer.decode(word_tokens))
                    word_tokens = [tokenized_inputs[i]]
                    predictions_words.append(first_pred)
                    first_pred = predictions[i]
                else:
                    word_tokens.append(tokenized_inputs[i])      
                previous_word_idx = word_idx

            words_res_x.extend(sent_words[1:])
            y_res_x.extend(predictions_words[1:])

        words_res.append(words_res_x)
        y_res.append(y_res_x)

    return words_res, y_res

In [48]:
def indexize(k=8):
    return "ID_" + "".join(random.choices(string.ascii_uppercase, k=k))

In [49]:
words_post_net = get_word_predictions(model, tokenizer, text)

In [50]:
output = []
entities = {}
persons = ["B-PER", "I-PER"]
locations = ["B-LOC", "I-LOC"]
for i, txt in enumerate(text):
    output.append("")
    words_entities_pairs = list(zip(words_post_net[0][i], words_post_net[1][i]))
    for pair in words_entities_pairs:
        # replace entity with an identifier
        if pair[1] in persons or pair[1] in locations:
            idx = indexize()
            while idx in entities:
                idx = indexize()
            word = idx
            entities[idx] = pair[0]
        else:
            word = pair[0]

        # add a word (anonymized or otherwise) to the output
        if word in string.punctuation or len(output[i]) == 0:
            output[i] += word
        else:
            output[i] += " " + word            

In [51]:
output

['« У селі жили і поляки, і українці. Мамина рідна сестра вийшла заміж за поляка. Там вони живуть досі ті мої двоюрідні сестри. Батькові сестри також повиходили заміж за поляків. Поки війни не було, то[ міжетнічної] різниці не було » – зазначає ID_HYOTWUHT ID_PZJLYMRB із с. Княгинінок.',
 'Про свій досвід взаємодії з євреями ID_JTMWOMGL ID_KFYRAHYV із с. Зелений ID_PQZMVBHG пригадує наступне: « Жиди приходили до нас додому. Мама хотіли їм квасолі дати, а він каже « ми квасолі не їмо ». Це так той єврей сказав. А чому він приходив, то я не знаю, мабуть через якусь потребу. Такі ж люди були, як і ми ... »',
 'Потім він по- польськи сказав “ Nemá ni<unk> ” » – розповідає ID_IAFOUGHU ID_OXAKLUOL з с. Коршів ID_UTMXZWXN ID_AXVOUTSP',
 'Інший календар, яким часто послуговуються оповідачі – релігійний. Оскільки більшість етнічних українців ID_ITPTKBZH належать до православної конфесії, найважливішими святами для них є Різдво( 7 січня), Водохреща( 19 січня), Стрітення( 15 лютого), Благовіщеня(

In [55]:
ner = pipeline('ner', model=model, tokenizer=tokenizer)
for txt in text:
    print(ner(txt))

[{'entity': 'B-PER', 'score': np.float32(0.9999862), 'index': 74, 'word': '▁Ган', 'start': 240, 'end': 244}, {'entity': 'B-PER', 'score': np.float32(0.9999864), 'index': 75, 'word': 'на', 'start': 244, 'end': 246}, {'entity': 'I-PER', 'score': np.float32(0.9999951), 'index': 76, 'word': '▁Ку', 'start': 246, 'end': 249}, {'entity': 'I-PER', 'score': np.float32(0.9999951), 'index': 77, 'word': 'хар', 'start': 249, 'end': 252}, {'entity': 'I-PER', 'score': np.float32(0.99999523), 'index': 78, 'word': 'чук', 'start': 252, 'end': 255}]
[{'entity': 'B-PER', 'score': np.float32(0.9999908), 'index': 9, 'word': '▁Антон', 'start': 35, 'end': 41}, {'entity': 'B-PER', 'score': np.float32(0.99999), 'index': 10, 'word': 'іна', 'start': 41, 'end': 44}, {'entity': 'I-PER', 'score': np.float32(0.9999956), 'index': 11, 'word': '▁Гу', 'start': 44, 'end': 47}, {'entity': 'I-PER', 'score': np.float32(0.9999958), 'index': 12, 'word': 'к', 'start': 47, 'end': 48}, {'entity': 'B-PER', 'score': np.float32(0.99